# LOAD DATA

In [7]:
# Read data
import pickle
def load_file(filename):
    with open(filename, 'rb') as fi:
        data = pickle.load(fi)
    return data

In [ ]:
from Data_Preprocessing import general as dp
from Data_Preprocessing import calculate_y as cy
from Data_Preprocessing import Train_test

filename = 'C:/Users/USER/Desktop/AIML/資料/Data'
data = load_file(filename)

data = data[data.isna()['UNIT_POST_CODE'] == False]
data = data[data.isna()['CITY'] == False]
data = data[data.isna()['DISTRICT'] == False]
data = data[data.isna()['LOCATION_ID'] == False]

# Training
data = dp.fillnamean(data)  
data = dp.drop_isable(data)
data = dp.drop_repeatcolumn(data)
data = dp.encoding_onehot(data)
data['START_DATE'] = dp.date2months(data['START_DATE'], 1950)
# data = data.drop(['START_DATE'], axis = 1)
data_train, data_test = dp.split_traintest(data, 0.2)
data_X, data_y = dp.separate_Xy(data_train)
data_Xtest, data_ytest = dp.separate_Xy(data_test)
#outlier_pred, data_X, data_y = dp.outlier_Isolation(data_X, data_y)#another outlier method
outlier_pred, data_X, data_y = dp.outlier_std(data_X, data_y, 4)
data_y = cy.calculate_ymean(data_y)
data_ytest = cy.calculate_ymean(data_ytest)


In [143]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler().fit(data_X)
X_scaled = scaler.transform(data_X)
Xtest_scaled = scaler.transform(data_Xtest)

# NEURAL NET

In [2]:
import sys
assert sys.version_info >= (3, 7)

from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")

### MLPRegressor

In [25]:
# Perceptron in sklearn?
from sklearn.metrics import mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

mlp_reg = MLPRegressor(hidden_layer_sizes=[10, 3, 2, 6], random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_reg)
pipeline.fit(data_X, data_y['y_mean'])
y_pred = pipeline.predict(data_X)
rmse_train = mean_absolute_error(data_y['y_mean'], y_pred)#, squared=False
y_pred = pipeline.predict(data_Xtest)
rmse_test = mean_absolute_error(data_ytest['y_mean'], y_pred)


c:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [14]:
# Housing data X_train.shape

(11610, 8)

### Sequential

In [144]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
X_train, X_valid, y_train, y_valid = train_test_split(
    X_scaled, data_y, random_state=42)

In [145]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    # norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

epoch = 50
loss_old = 100000000
lr = 1
for i in range(epoch):
    print(f"--------------------epoch:{i}----------------------------")
    print("lr:", lr)
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss="mae", optimizer=optimizer, metrics=["RootMeanSquaredError"])
        # -------------Question------------------
        # norm_layer.adapt(X_train)
        # what does adapt mean
        # ---------------------------------------
        # norm_layer.adapt(X_train)
    history = model.fit(X_train, y_train['y_mean'], validation_data=(X_valid, y_valid))
    if history.history['val_loss'][0] <= loss_old:
        model.save('better_model')
        print('-----good-----')
        loss_old = history.history['val_loss'][0]
        lr*=1.2
        # -------------Question------------------
        # mse_test, rmse_test = model.evaluate(data_Xtest, data_ytest['y_mean'])
        # what does evaluate mean
        # ---------------------------------------
    else:
        model = keras.models.load_model('better_model')  
        print('-----bad-----')
        lr*=0.8
        if lr<=1e-6:
            break
    #y_pred = model.predict(data_Xtest)
    #mae_test = mean_absolute_error(data_ytest['y_mean'], y_pred)


--------------------epoch:0----------------------------
lr: 1
221/221 [==============================] - 1s 2ms/step - loss: 59262.3750 - root_mean_squared_error: 197859.7500 - val_loss: 70820.6250 - val_root_mean_squared_error: 164418.3594
INFO:tensorflow:Assets written to: better_model\assets
-----good-----
--------------------epoch:1----------------------------
lr: 1.2
221/221 [==============================] - 1s 2ms/step - loss: 55481.6602 - root_mean_squared_error: 196149.0625 - val_loss: 70166.9531 - val_root_mean_squared_error: 164110.5781
INFO:tensorflow:Assets written to: better_model\assets
-----good-----
--------------------epoch:2----------------------------
lr: 1.44
221/221 [==============================] - 1s 2ms/step - loss: 63583.8984 - root_mean_squared_error: 200185.5156 - val_loss: 70823.0234 - val_root_mean_squared_error: 168613.9844
-----bad-----
--------------------epoch:3----------------------------
lr: 1.152
221/221 [==============================] - 1s 2ms/st

KeyboardInterrupt: 

In [ ]:
model = keras.models.load_model('better_model')
y_pred = model.predict(data_Xtest)
mae_test = mean_absolute_error(data_ytest['y_mean'], y_pred)
mae_test